In [ ]:
from nbdev import *
default_exp evolver

In [ ]:
#hide
import sys
sys.path.append("..")
import json
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
from dpct.individual import DHPCTIndividual

# DHPCTEvolver

> Class for evolving populations of DHPCTIndividual instances using DEAP.

In [ ]:
#export
class DHPCTEvolver:
    """DHPCTEvolver provides evolutionary capabilities for populations of DHPCTIndividual instances."""
    
    def __init__(self, population_size=20, gens=50, cx_prob=0.7, mut_prob=0.2, elite_size=1, env_template=None):
        """
        Initialize a new evolver with evolution parameters.
        
        Parameters:
        - population_size: Number of individuals in the population
        - gens: Number of generations to evolve
        - cx_prob: Crossover probability
        - mut_prob: Mutation probability
        - elite_size: Number of elite individuals to preserve
        - env_template: Template DHPCTIndividual to use for creating population
        """
        self.population_size = population_size
        self.gens = gens
        self.cx_prob = cx_prob
        self.mut_prob = mut_prob
        self.elite_size = elite_size
        self.env_template = env_template
        self.toolbox = None
        self.stats = None
        self.logbook = None
        self.best_individual = None
        self.final_population = None
        
    def setup_evolution(self, evaluation_episodes=5, mutation_rate=0.1):
        """
        Set up the DEAP toolbox for evolution.
        
        Parameters:
        - evaluation_episodes: Number of episodes for individual evaluation
        - mutation_rate: Rate of mutation in individual properties
        """
        # Create fitness and individual types
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)
        
        # Initialize toolbox
        self.toolbox = base.Toolbox()
        
        # Define attribute generator (DHPCTIndividual cloning)
        def create_individual():
            if self.env_template is None:
                raise ValueError("env_template must be provided")
            # Create a clone of the template with possibly mutated properties
            config = self.env_template.config()
            individual = DHPCTIndividual.from_config(config)
            return individual
        
        # Register individual creation
        self.toolbox.register("individual_creator", create_individual)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.individual_creator, 1)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        # Define evaluation function
        def evaluate_individual(individual):
            # Extract the DHPCTIndividual from the list wrapper
            dhpct_individual = individual[0]
            # Evaluate the individual
            fitness = dhpct_individual.evaluate(episodes=evaluation_episodes)
            return (fitness,)  # Return as tuple for DEAP
        
        # Define mating function
        def mate_individuals(ind1, ind2):
            # Extract DHPCTIndividuals
            dhpct_ind1 = ind1[0]
            dhpct_ind2 = ind2[0]
            # Create offspring through mating
            offspring = dhpct_ind1.mate(dhpct_ind2)
            # Replace the content with the new offspring
            ind1[0] = offspring
            return ind1,
        
        # Define mutation function
        def mutate_individual(individual):
            # Extract and mutate the DHPCTIndividual
            dhpct_individual = individual[0]
            dhpct_individual.mutate(mutation_rate=mutation_rate)
            return individual,
        
        # Register genetic operators
        self.toolbox.register("evaluate", evaluate_individual)
        self.toolbox.register("mate", mate_individuals)
        self.toolbox.register("mutate", mutate_individual)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Set up statistics
        self.stats = tools.Statistics(lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)
        
        self.logbook = tools.Logbook()
        
    def run_evolution(self, verbose=True):
        """
        Run the evolutionary process.
        
        Parameters:
        - verbose: Whether to print progress
        
        Returns:
        - Final population, logbook with statistics
        """
        if self.toolbox is None:
            self.setup_evolution()
            
        # Initialize population
        pop = self.toolbox.population(n=self.population_size)
        
        # Evaluate initial population
        fitnesses = list(map(self.toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
            
        # Record statistics for initial population
        record = self.stats.compile(pop)
        self.logbook.record(gen=0, **record)
        if verbose:
            print(self.logbook.stream)
            
        # Evolution loop
        for gen in range(1, self.gens+1):
            # Select elite individuals
            elites = tools.selBest(pop, self.elite_size)
            elites = list(map(toolbox.clone, elites))  # Make deep copies
            
            # Select individuals for breeding
            offspring = self.toolbox.select(pop, len(pop) - self.elite_size)
            offspring = list(map(toolbox.clone, offspring))
            
            # Apply crossover
            for i in range(1, len(offspring), 2):
                if i < len(offspring) and np.random.random() < self.cx_prob:
                    self.toolbox.mate(offspring[i-1], offspring[i])
                    # Clear fitness values of modified individuals
                    del offspring[i-1].fitness.values
                    if i < len(offspring):
                        del offspring[i].fitness.values
                        
            # Apply mutation
            for i in range(len(offspring)):
                if np.random.random() < self.mut_prob:
                    self.toolbox.mutate(offspring[i])
                    # Clear fitness values of modified individuals
                    del offspring[i].fitness.values
                    
            # Add elites back to population
            offspring.extend(elites)
            
            # Evaluate new individuals (those without fitness values)
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                
            # Replace population
            pop[:] = offspring
            
            # Record statistics
            record = self.stats.compile(pop)
            self.logbook.record(gen=gen, **record)
            if verbose:
                print(self.logbook.stream)
                
        # Store results
        self.final_population = pop
        self.best_individual = tools.selBest(pop, 1)[0][0]  # Extract DHPCTIndividual
        
        return pop, self.logbook
    
    def save_results(self, population_file="population.json", logbook_file="logbook.json", best_file="best_individual.json"):
        """
        Save evolution results to files.
        
        Parameters:
        - population_file: File to save final population configurations
        - logbook_file: File to save evolution logbook
        - best_file: File to save best individual configuration
        """
        if self.final_population is None:
            raise ValueError("No evolution results to save. Run evolution first.")
            
        # Save population
        population_configs = [ind[0].config() for ind in self.final_population]
        with open(population_file, 'w') as f:
            json.dump(population_configs, f, indent=2)
            
        # Save logbook (convert to serializable format)
        logbook_data = {
            'generations': list(range(len(self.logbook))),
            'avg': self.logbook.select('avg'),
            'min': self.logbook.select('min'),
            'max': self.logbook.select('max'),
            'std': self.logbook.select('std')
        }
        with open(logbook_file, 'w') as f:
            json.dump(logbook_data, f, indent=2)
            
        # Save best individual
        if self.best_individual is not None:
            self.best_individual.save_config(best_file)
            
    def plot_evolution(self):
        """
        Plot the evolution progress.
        """
        if self.logbook is None:
            raise ValueError("No evolution data to plot. Run evolution first.")
            
        gen = self.logbook.select("gen")
        fit_avg = self.logbook.select("avg")
        fit_max = self.logbook.select("max")
        fit_min = self.logbook.select("min")
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(gen, fit_avg, label="Average")
        ax.plot(gen, fit_max, label="Max")
        ax.plot(gen, fit_min, label="Min")
        ax.set_xlabel("Generation")
        ax.set_ylabel("Fitness")
        ax.set_title("Evolution Progress")
        ax.legend()
        ax.grid(True)
        
        return fig

In [ ]:
# Example of using DHPCTEvolver# Create a template individualtemplate = DHPCTIndividual(env_name="CartPole-v1",    env_props={},    levels=[        {'units': 10},        {'units': 5}    ],    activation_funcs=['relu', 'tanh'],    weight_types=['glorot_uniform', 'glorot_uniform'])# Initialize the evolverevolver = DHPCTEvolver(    population_size=10,  # Small size for quick example    gens=5,              # Few generations for quick example    cx_prob=0.7,    mut_prob=0.2,    elite_size=1,    env_template=template)

In [ ]:
# Setup evolution
evolver.setup_evolution(evaluation_episodes=2, mutation_rate=0.1)

# Run evolution (commented out as it would take time)
# pop, log = evolver.run_evolution(verbose=True)

In [ ]:
from nbdev import *
default_exp evolver

In [ ]:
#hide
import sys
sys.path.append("..")
import json
import time
import random
import numpy as np
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

In [ ]:
#hide
from dpct.individual import DHPCTIndividual

# DHPCTEvolver

> Class for evolving Deep Perceptual Control Theory individuals using DEAP.

In [ ]:
#export
class DHPCTEvolver:
    """DHPCTEvolver configures and runs evolutionary optimization of DHPCTIndividual populations using DEAP."""
    
    def __init__(self, 
                 pop_size=50, 
                 generations=100,
                 evolve_termination=None,
                 evolve_static_termination=None,
                 unchanged_generations=5,
                 run_best=True,
                 save_arch_best=True,
                 save_arch_all=False):
        """
        Initialize the evolver with configuration parameters.
        
        Parameters:
        - pop_size: Size of the population
        - generations: Maximum number of generations
        - evolve_termination: Fitness target for termination
        - evolve_static_termination: Whether to terminate if no improvement
        - unchanged_generations: Number of generations without improvement before termination
        - run_best: Whether to evaluate and display the best individual each generation
        - save_arch_best: Whether to save the best individual each generation
        - save_arch_all: Whether to save all individuals each generation
        """
        self.pop_size = pop_size
        self.generations = generations
        self.evolve_termination = evolve_termination
        self.evolve_static_termination = evolve_static_termination
        self.unchanged_generations = unchanged_generations
        self.run_best = run_best
        self.save_arch_best = save_arch_best
        self.save_arch_all = save_arch_all
        
        self.toolbox = None
        self.stats = None
        self.logbook = None
        self.hof = None
        self.population = None
        self.best_fitness = -float('inf')  # For maximization
        self.best_gen = 0
        self.best_individual = None
        
    def setup_evolution(self, template_individual, fitness_function, minimize=True):
        """Configure DEAP toolbox with evolutionary operators
        
        Parameters:
        - template_individual: An instance of DHPCTIndividual to use as a template
        - fitness_function: Function to evaluate individuals
        - minimize: Whether to minimize or maximize the fitness function
        """
        # Create fitness and individual classes
        if minimize:
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
            creator.create("Individual", DHPCTIndividual, fitness=creator.FitnessMin)
            self.best_fitness = float('inf')  # Start with worst possible for minimization
        else:
            creator.create("FitnessMax", base.Fitness, weights=(1.0,))
            creator.create("Individual", DHPCTIndividual, fitness=creator.FitnessMax)
            self.best_fitness = -float('inf')  # Start with worst possible for maximization
        
        # Create toolbox
        self.toolbox = base.Toolbox()
        
        # Register initialization function
        def init_individual():
            # Create a copy of the template individual
            return creator.Individual(
                template_individual.env_name,
                template_individual.gym_name,
                template_individual.env_props.copy(),
                template_individual.levels[:],
                template_individual.activation_funcs.copy(),
                template_individual.weight_types.copy()
            )
        
        self.toolbox.register("individual", init_individual)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        # Register genetic operators
        self.toolbox.register("mate", self._mate)
        self.toolbox.register("mutate", self._mutate)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        # Register evaluation function
        self.toolbox.register("evaluate", fitness_function)
        
        # Set up statistics
        self.stats = tools.Statistics(lambda ind: ind.fitness.values[0])
        self.stats.register("min", np.min)
        self.stats.register("avg", np.mean)
        self.stats.register("max", np.max)
        self.stats.register("std", np.std)
        
        # Hall of fame to track the best individual
        self.hof = tools.HallOfFame(1)
        
        return self
    
    def _mate(self, ind1, ind2):
        """Crossover operator for DEAP"""
        child1, child2 = ind1.mate(ind2)
        # Convert to creator.Individual class
        child1_creator = creator.Individual(
            child1.env_name,
            child1.gym_name,
            child1.env_props,
            child1.levels,
            child1.activation_funcs,
            child1.weight_types
        )
        
        child2_creator = creator.Individual(
            child2.env_name,
            child2.gym_name,
            child2.env_props,
            child2.levels,
            child2.activation_funcs,
            child2.weight_types
        )
        
        return child1_creator, child2_creator
    
    def _mutate(self, individual):
        """Mutation operator for DEAP"""
        individual.mutate()
        return individual,
    
    def run_evolution(self, verbose=True):
        """Run the evolutionary process
        
        Parameters:
        - verbose: Whether to print progress information
        
        Returns:
        - The final population
        - The logbook with statistics
        - The hall of fame containing the best individual
        """
        if self.toolbox is None:
            raise ValueError("Evolution not set up. Call setup_evolution first.")
        
        # Initialize population
        self.population = self.toolbox.population(n=self.pop_size)
        self.logbook = tools.Logbook()
        self.logbook.header = ['gen', 'nevals'] + self.stats.fields
        
        # Evaluate initial population
        invalid_ind = [ind for ind in self.population if not ind.fitness.valid]
        fitnesses = self.toolbox.map(self.toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = (fit,)
        
        # Update hall of fame
        self.hof.update(self.population)
        
        # Record statistics
        record = self.stats.compile(self.population)
        self.logbook.record(gen=0, nevals=len(invalid_ind), **record)
        
        # Check if we have a new best
        self._update_best(0, record)
        
        if verbose:
            print(self.logbook.stream)
        
        # Begin evolution
        for gen in range(1, self.generations + 1):
            # Start generation timing
            gen_start_time = time.time()
            
            # Select next generation
            offspring = self.toolbox.select(self.population, len(self.population))
            offspring = list(map(self.toolbox.clone, offspring))
            
            # Apply crossover
            for i in range(1, len(offspring), 2):
                if random.random() < 0.5:  # 50% chance of crossover
                    offspring[i-1], offspring[i] = self.toolbox.mate(offspring[i-1], offspring[i])
                    del offspring[i-1].fitness.values
                    del offspring[i].fitness.values
            
            # Apply mutation
            for i in range(len(offspring)):
                if random.random() < 0.2:  # 20% chance of mutation
                    offspring[i], = self.toolbox.mutate(offspring[i])
                    del offspring[i].fitness.values
            
            # Evaluate invalid individuals
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = self.toolbox.map(self.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = (fit,)
            
            # Replace population
            self.population[:] = offspring
            
            # Update hall of fame
            self.hof.update(self.population)
            
            # Record statistics
            record = self.stats.compile(self.population)
            self.logbook.record(gen=gen, nevals=len(invalid_ind), **record)
            
            # Check if we have a new best
            improved = self._update_best(gen, record)
            
            # End generation timing
            gen_time = time.time() - gen_start_time
            
            if verbose:
                print(f"{self.logbook.stream} (time: {gen_time:.2f}s)")
            
            # Run best individual if requested
            if self.run_best:
                best_ind = self.hof[0]
                best_fit = best_ind.evaluate()
                if verbose:
                    print(f"Best individual fitness: {best_fit}")
            
            # Save configurations if requested
            if self.save_arch_best:
                best_ind = self.hof[0]
                best_ind.save_config(f"best_individual_gen_{gen}.json")
            
            if self.save_arch_all:
                for i, ind in enumerate(self.population):
                    ind.save_config(f"individual_{i}_gen_{gen}.json")
            
            # Check termination conditions
            if self.evolve_termination is not None:
                if (record["min" if "FitnessMin" in str(creator.Individual().fitness.__class__) else "max"] <= 
                        self.evolve_termination):
                    if verbose:
                        print(f"Termination condition reached at generation {gen}")
                    break
            
            if self.evolve_static_termination and not improved:
                if gen - self.best_gen >= self.unchanged_generations:
                    if verbose:
                        print(f"No improvement for {self.unchanged_generations} generations. Terminating.")
                    break
        
        # Save the best individual for later use
        self.best_individual = self.hof[0]
        
        return self.population, self.logbook, self.hof
    
    def _update_best(self, gen, record):
        """Update the best fitness and generation"""
        is_min = "FitnessMin" in str(creator.Individual().fitness.__class__)
        current = record["min" if is_min else "max"]
        
        if (is_min and current < self.best_fitness) or (not is_min and current > self.best_fitness):
            self.best_fitness = current
            self.best_gen = gen
            return True
        
        return False
    
    def save_results(self, path):
        """Save evolution results and configurations
        
        Parameters:
        - path: Base path to save results to
        """
        # Save statistics
        with open(f"{path}_stats.json", "w") as f:
            json.dump(self.logbook, f, indent=2)
        
        # Save best individual
        if self.best_individual:
            self.best_individual.save_config(f"{path}_best_individual.json")
        
        # Generate plots
        self._generate_plots(path)
    
    def _generate_plots(self, path):
        """Generate and save plots of the evolution progress"""
        if self.logbook is None:
            return
        
        gen = self.logbook.select("gen")
        fit_mins = self.logbook.select("min")
        fit_avgs = self.logbook.select("avg")
        fit_maxs = self.logbook.select("max")
        
        plt.figure(figsize=(10, 6))
        plt.plot(gen, fit_mins, "b-", label="Minimum Fitness")
        plt.plot(gen, fit_avgs, "g-", label="Average Fitness")
        plt.plot(gen, fit_maxs, "r-", label="Maximum Fitness")
        plt.xlabel("Generation")
        plt.ylabel("Fitness")
        plt.title("Evolution Progress")
        plt.legend(loc="best")
        plt.grid(True)
        plt.savefig(f"{path}_fitness_plot.png")
        plt.close()

In [ ]:
# Test creating an evolver
evolver = DHPCTEvolver(pop_size=10, generations=5)